In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os, sys # add MaaSSim to path (not needed if MaaSSim is already in path)
module_path = os.path.abspath(os.path.join('../..'))
if module_path not in sys.path:
    sys.path.append(module_path)
import MaaSSim

In [3]:
#!pip install MaaSSim
import MaaSSim.utils
import logging
from dotmap import DotMap
import MaaSSim.simulators
from MaaSSim.data_structures import structures as inData
from MaaSSim.acceptance import f_decline, f_decline_R50, f_decline_R75, f_decline_R100, f_decline_mixed
from MaaSSim.acceptance import RA_kpi_veh, RA_kpi_pax

In [4]:
import h3
import folium
import pandas as pd
import matplotlib.pyplot as plt
import osmnx as ox
import networkx as nx
import random
import pandas as pd
import numpy as np
import seaborn as sns

In [7]:
params = MaaSSim.utils.get_config('glance.json')
params.vehicle_fixed_positions = True
params.times.patience = 600 #in second
params.dist_threshold = 20000 #in meter
params.dist_threshold_min = 3000
params.city = "Amsterdam, Netherlands"
params.paths.G = "../../data/graphs/Amsterdam.graphml"
params.paths.skim = "../../data/graphs/Amsterdam.csv"

params.d2d.fuel_cost = 0.25 #euro/km
params.platforms.comm_rate = 0
params.platforms.base_fare = 2
params.platforms.fare = 1.2 
params.platforms.min_fare = 2
params.speeds.ride=10
params.times.decline=20
params.assert_me = False

params.paths.requests = 'Amsterdam_requests_new.csv'
params.zoning_level = 7

params.nP = 10
params.nV = 1
params.simTime = 4

In [8]:
sim = MaaSSim.simulators.simulate(params=params)

08-09-22 10:24:10-WARNING-Setting up 4h simulation at 2022-09-08 08:28:04 for 1 vehicles and 10 passengers in Amsterdam, Netherlands
08-09-22 10:24:14-INFO--------------------	Starting simulation	-------------------
               hex7  cnt  demand  supply  D/S  surge_mp
0   871968240ffffff   11       0       0  NaN       NaN
1   871968241ffffff   12       0       0  NaN       NaN
2   871968244ffffff   12       0       0  NaN       NaN
3   871968245ffffff   30       0       0  NaN       NaN
4   871968248ffffff   34       0       0  NaN       NaN
5   871968249ffffff  250       0       0  NaN       NaN
6   87196824cffffff   47       0       0  NaN       NaN
7   87196824dffffff  346       0       0  NaN       NaN
8   871968263ffffff    1       0       0  NaN       NaN
9   871968268ffffff   38       0       0  NaN       NaN
10  871968269ffffff  461       0       0  NaN       NaN
11  87196826affffff  122       0       0  NaN       NaN
12  87196826bffffff  562       0       0  NaN       NaN


In [9]:
def visualize_hexagons(hexagons, color="red", w=1, folium_map=None):
    """
    hexagons is a list of hexcluster. Each hexcluster is a list of hexagons. 
    eg. [[hex1, hex2], [hex3, hex4]]
    """
    polylines = []
    lat = []
    lng = []
    for hex in hexagons:
        polygons = h3.h3_set_to_multi_polygon([hex], geo_json=False)
        # flatten polygons into loops.
        outlines = [loop for polygon in polygons for loop in polygon]
        polyline = [outline + [outline[0]] for outline in outlines][0]
        lat.extend(map(lambda v:v[0],polyline))
        lng.extend(map(lambda v:v[1],polyline))
        polylines.append(polyline)
    
    if folium_map is None:
        m = folium.Map(location=[sum(lat)/len(lat), sum(lng)/len(lng)], zoom_start=13, tiles='cartodbpositron')
    else:
        m = folium_map
    for polyline in polylines:
        my_PolyLine=folium.PolyLine(locations=polyline,weight=w,color=color,fill= True)
        m.add_child(my_PolyLine)
    return m
    

def visualize_polygon(polyline, color):
    polyline.append(polyline[0])
    lat = [p[0] for p in polyline]
    lng = [p[1] for p in polyline]
    m = folium.Map(location=[sum(lat)/len(lat), sum(lng)/len(lng)], zoom_start=13, tiles='cartodbpositron')
    my_PolyLine=folium.PolyLine(locations=polyline,weight=8,color=color)
    m.add_child(my_PolyLine)
    return m

In [8]:
paths= "../../data/graphs/Amsterdam.graphml"
G = ox.load_graphml(paths)

In [10]:
G = sim.inData.G

In [15]:
# Visualiziation of hexagons on Amsterdam map
lat = []
lng = []
for i in G.nodes:
    lat.append(G.nodes[i]['y'])
    lng.append(G.nodes[i]['x'])
    
df = pd.DataFrame()
df['lat'] = lat
df['lng'] = lng

level = 5
hex_col = 'hex'+str(level)
df[hex_col] = df.apply(lambda x: h3.geo_to_h3(x.lat,x.lng,level),axis=1)
df = df.groupby(hex_col).size().to_frame('cnt').reset_index()

l = list(df[hex_col])
m = visualize_hexagons(l)
display(m)

In [13]:
df

,hex7,cnt
0,871968240ffffff,11
1,871968241ffffff,12
2,871968244ffffff,12
3,871968245ffffff,30
4,871968248ffffff,34
5,871968249ffffff,250
6,87196824cffffff,47
7,87196824dffffff,346
8,871968263ffffff,1
9,871968268ffffff,38


In [14]:
sim.requests

,pax_id,origin,destination,treq,tdep,ttrav,tarr,tdrop,shareable,schedule_id,dist,ride_id,position,sim_schedule
id,,,,,,,,,,,,,,
1,403,5604910198,46284575,2022-08-19 05:06:16,NaN,0 days 00:07:19,2022-08-19 05:13:35,NaN,False,NaN,4395,403,0,node time req_id od 0 ...
2,159,46340766,503322356,2022-08-19 05:06:17,NaN,0 days 00:12:54,2022-08-19 05:19:11,NaN,False,NaN,7745,159,0,node time req_id od 0 N...
3,369,413562329,46423248,2022-08-19 05:06:34,NaN,0 days 00:13:40,2022-08-19 05:20:14,NaN,False,NaN,8200,369,0,node time req_id od 0 N...
4,287,7997957408,3297465151,2022-08-19 05:06:45,NaN,0 days 00:08:06,2022-08-19 05:14:51,NaN,False,NaN,4865,287,0,node time req_id od 0 ...
5,236,46354394,46286157,2022-08-19 05:06:46,NaN,0 days 00:07:13,2022-08-19 05:13:59,NaN,False,NaN,4338,236,0,node time req_id od 0 NaN...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
996,396,969196812,46464099,2022-08-19 09:04:24,NaN,0 days 00:11:23,2022-08-19 09:15:47,NaN,False,NaN,6831,396,0,node time req_id od 0 N...
997,221,3509181973,46471328,2022-08-19 09:05:19,NaN,0 days 00:08:32,2022-08-19 09:13:51,NaN,False,NaN,5128,221,0,node time req_id od 0 ...
998,614,46550991,46377676,2022-08-19 09:05:31,NaN,0 days 00:14:57,2022-08-19 09:20:28,NaN,False,NaN,8971,614,0,node time req_id od 0 NaN...


In [24]:
req = sim.requests
# req = req[sim.t0<=req.treq]
# req = req[req.treq<sim.t0 + pd.Timedelta(60, 's')]

level = 7
df['demand'] = 0 
df['supply'] = 0 

for index, row  in req.iterrows():
    lat = G.nodes[row.origin]['y']
    lng = G.nodes[row.origin]['x']
    
    add = h3.geo_to_h3(lat,lng,level)
    
    index = df[df.hex7 == add].index
    df.at[index,'demand'] = df.loc[index].demand+1
    
for index, row  in sim.vehicles.iterrows():
    lat = G.nodes[row.pos]['y']
    lng = G.nodes[row.pos]['x']
    
    add = h3.geo_to_h3(lat,lng,level)
    
    index = df[df.hex7 == add].index
    df.at[index,'supply'] = df.loc[index].supply+1

df['D/S'] = df.demand/df.supply
df

,hex7,cnt,demand,supply,D/S
0,871968240ffffff,11,0,0,NaN
1,871968241ffffff,12,5,1,5.000000
2,871968244ffffff,12,1,0,inf
3,871968245ffffff,30,0,0,NaN
4,871968248ffffff,34,1,0,inf
5,871968249ffffff,250,10,1,10.000000
6,87196824cffffff,47,5,0,inf
7,87196824dffffff,346,21,1,21.000000
8,871968263ffffff,1,0,0,NaN
9,871968268ffffff,38,1,0,inf


In [18]:
vehs = sim.vehicles

In [19]:
pax = sim.passengers

In [25]:
level = 7
df['demand'] = 0 
df['supply'] = 0 

for index, row  in pax.iterrows():
    lat = G.nodes[row.pos]['y']
    lng = G.nodes[row.pos]['x']
    
    add = h3.geo_to_h3(lat,lng,level)
    
    index = df[df.hex7 == add].index
    df.at[index,'demand'] = df.loc[index].demand+1
    
for index, row  in vehs.iterrows():
    lat = G.nodes[row.pos]['y']
    lng = G.nodes[row.pos]['x']
    
    add = h3.geo_to_h3(lat,lng,level)
    
    index = df[df.hex7 == add].index
    df.at[index,'supply'] = df.loc[index].supply+1

df['D/S'] = df.demand/df.supply
df

,hex7,cnt,demand,supply,D/S
0,871968240ffffff,11,0,0,NaN
1,871968241ffffff,12,3,1,3.000000
2,871968244ffffff,12,1,0,inf
3,871968245ffffff,30,0,0,NaN
4,871968248ffffff,34,0,0,NaN
5,871968249ffffff,250,9,1,9.000000
6,87196824cffffff,47,4,0,inf
7,87196824dffffff,346,15,1,15.000000
8,871968263ffffff,1,0,0,NaN
9,871968268ffffff,38,3,0,inf


In [21]:
vehs

,pos,event,shift_start,shift_end,platform,expected_income
veh,,,,,,
1,46367433,driverEvent.ENDS_SHIFT,0,86400,1,NaN
2,46306612,driverEvent.ENDS_SHIFT,0,86400,1,NaN
3,297970323,driverEvent.ENDS_SHIFT,0,86400,1,NaN
4,46360124,driverEvent.ENDS_SHIFT,0,86400,1,NaN
5,1420479319,driverEvent.ENDS_SHIFT,0,86400,1,NaN
...,...,...,...,...,...,...
96,372972696,driverEvent.ENDS_SHIFT,0,86400,1,NaN
97,506965977,driverEvent.ENDS_SHIFT,0,86400,1,NaN
98,46418111,driverEvent.ENDS_SHIFT,0,86400,1,NaN
